<a href="https://colab.research.google.com/github/harman-ransi-1016/AI_News_Detection/blob/main/CNN_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import CNN news dataset

In [ ]:
import pandas as pd
df = pd.read_csv('..\Data\CNN_Articels_clean\CNN_Articels_clean.csv')
df2 = pd.read_csv('..\Data\CNN_Articels_clean_2\CNN_Articels_clean.csv')

In [ ]:
frames = [df, df2]
news = pd.concat(frames)
len(news)

42025

In [ ]:
news.head(1)

,Index,Author,Date published,Category,Section,Url,Headline,Description,Keywords,Second headline,Article text
0,0,"Jacopo Prisco, CNN",2021-07-15 02:46:59,news,world,https://www.cnn.com/2021/07/14/world/tusimple-...,"There's a shortage of truckers, but TuSimple t...",The e-commerce boom has exacerbated a global t...,"world, There's a shortage of truckers, but TuS...","There's a shortage of truckers, but TuSimple t...","(CNN)Right now, there's a shortage of truck d..."


In [ ]:
news['Date published'] = pd.to_datetime(news['Date published'], errors='coerce')
news = news.dropna()
len(news)

42016

In [ ]:
news = news[news['Date published'].dt.year <= 2020]
len(news)

29435

In [ ]:
news

,Index,Author,Date published,Category,Section,Url,Headline,Description,Keywords,Second headline,Article text
119,144,"Noura Abou Zeinab, for CNN",2020-12-02 09:25:41,sport,sport,https://www.cnn.com/2020/12/02/sport/rene-camp...,Bodybuilding: Rene Campbell wants to change pe...,Counterpointed by a number of striking tattoos...,"sport, Bodybuilding: Rene Campbell wants to ch...",Bodybuilder Rene Campbell wants to change the ...,(CNN)Counterpointed by a number of striking t...
120,145,"Noura Abou Zeinab, for CNN",2020-10-06 09:02:31,sport,sport,https://www.cnn.com/2020/10/06/sport/priscilla...,Eating disorders: Olympic high jumper Priscill...,"The rejection was brutal. At the age of 16, Pr...","sport, Eating disorders: Olympic high jumper P...",Olympic high jumper pressured to 'perform bett...,(CNN)The rejection was brutal. At the age of ...
121,146,"Noura Abou Zeinab, CNN",2020-06-10 09:37:16,sport,sport,https://www.cnn.com/2020/06/10/sport/marilyn-o...,Women sport stars' fight for body acceptance -...,"She was comfortable with her body, though othe...","sport, Women sport stars' fight for body accep...",'Oh you must be heavy boned.' Marilyn Okoro's ...,"(CNN)She was comfortable with her body, thoug..."
154,201,"Sarah Lazarus and Jon Jensen, CNN",2020-01-15 09:43:30,news,asia,https://www.cnn.com/2020/01/15/asia/anjali-wat...,"Sri Lanka's leopards are under threat, but thi...",With as few as 750 leopards left on the island...,"asia, Sri Lanka's leopards are under threat, b...","Sri Lanka's leopards are under threat, but thi...",(CNN)Having endured a bloody civil war from 1...
155,202,Amy Woodyatt,2020-01-07 17:38:23,news,asia,https://www.cnn.com/2020/01/07/asia/fukushima-...,Wildlife flourishing in uninhabited areas arou...,Nearly a decade after Japan's Fukushima nuclea...,"asia, Wildlife flourishing in uninhabited area...",Wildlife flourishing in uninhabited areas arou...,(CNN)Nearly a decade after Japan's Fukushima ...
...,...,...,...,...,...,...,...,...,...,...,...
37897,44933,CNN Editorial Research,2013-01-03 20:30:07,news,europe,https://www.cnn.com/2013/01/03/world/europe/vl...,Vladimir Putin Fast Facts - CNN,Read CNN's Fast Facts about Russian President ...,"europe, Vladimir Putin Fast Facts - CNN",Vladimir Putin Fast Facts,(CNN)Here's a look at the life of Russian Pre...
37907,44945,CNN Editorial Research,2012-12-27 22:48:19,news,europe,https://www.cnn.com/2012/12/27/world/europe/ma...,Mario Monti Fast Facts - CNN,View CNN's Mario Monti Fast Facts for a look a...,"europe, Mario Monti Fast Facts - CNN",Mario Monti Fast Facts,(CNN)Here is a look at the life of economist ...
37910,44948,CNN Editorial Research,2013-04-29 13:50:05,news,europe,https://www.cnn.com/2013/04/29/world/europe/ra...,Ratko Mladic Fast Facts - CNN,Read CNN's Ratko Mladic Fast Facts for a look ...,"europe, Ratko Mladic Fast Facts - CNN",Ratko Mladic Fast Facts,(CNN)Here is a look at the life of Ratko Mlad...
37947,44996,CNN Editorial Research,2013-01-12 01:42:49,news,europe,https://www.cnn.com/2013/01/11/world/europe/mi...,Mikhail Gorbachev Fast Facts - CNN,"Read CNN's Fast Facts on Mikhail Gorbachev, fo...","europe, Mikhail Gorbachev Fast Facts - CNN",Mikhail Gorbachev Fast Facts,Here's a look at the life of Mikhail Gorbachev...


In [ ]:
news = news[~news['Article text'].str.contains("MUST WATCH", case=False, na=False)]
news = news[~news['Article text'].str.contains("Hide Caption", case=False, na=False)]

In [ ]:
len(news)

15753

Create CSV for saving

In [ ]:
df_empty = pd.DataFrame(columns=news.columns)
df_empty['GPT3.5-Turbo'] = None
# Write the empty DataFrame to a CSV file
df_empty.to_csv('genArt.csv', index=False)

In [ ]:
import re

pattern_contributor = r"\b(?:CNN|BBC|ABC)'s \w+ \w+ contributed to this report\."
pattern_parentheses = r"\(CNN\)"
pattern_highlights = r"Story highlights"

# Define a function to modify the text
def modify_text(text):
    # Remove "(CNN)"
    text = re.sub(pattern_parentheses, '', text)
    # Add space after a period if missing
    text = re.sub(r'(?<=[^.])\.(?=[^\s])', '. ', text)
    text = re.sub(pattern_contributor, '', text)
    # Remove "Story highlights:"
    text = re.sub(pattern_highlights, '', text)
    return text

news['Article text'] = news['Article text'].apply(lambda x: modify_text(x))

In [ ]:
import re
def count_words(text):
    words = re.findall(r'\w+', text)
    return len(words)

In [ ]:
news = news[news['Article text'].apply(count_words) <= 500]

In [ ]:
len(news)

6631

In [ ]:
import tiktoken
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [ ]:
def getinput(dfin):
    article_length = count_words(dfin.iloc[0]['Article text'])
    input = "Title: " + dfin.iloc[0]['Headline'].split(" - ")[0] + "\nLength of Article (in words): " + str(article_length) + "\nDescription: " + dfin.iloc[0]['Description']
    return input

In [ ]:
getinput(news.sample())

'Title: Munroe Bergdorf reveals abuse she\'s received since L\'Oreal appointment\nLength of Article (in words): 381\nDescription: British model Munroe Bergdorf has said her social media messages have been "flooded" by racist and transphobic abuse since she announced she will join L\'Oreal\'s UK diversity board.'

In [ ]:
num_tokens_from_string("You are a tool for generating article content given a tile, word length, and short description. You should make the article as close as you can to the specified length,preferably lower. You should only return the content of the article. If for some reason you cannot complete the request return the response \"Cannot complete request\" ","gpt-3.5-turbo")

66

In [ ]:
samp = news.sample()
input = getinput(news.sample())
tok = 66 + num_tokens_from_string(input,"gpt-3.5-turbo") + num_tokens_from_string(samp.iloc[0]['Article text'], "gpt-3.5-turbo")
tok

468

In [ ]:
import openai
import os
from dotenv import load_dotenv
load_dotenv()
token = os.environ.get("api-token")
openai.api_key = token

response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo-16k-0613",
  messages=[
    {
      "role": "system",
      "content": "You are a tool for generating article content given a tile, word length, and short description. You should make the article as close as you can to the specified length,preferably lower. You should only return the content of the article. If for some reason you cannot complete the request return the response \"Cannot complete request\" "
    },
    {
      "role": "user",
      "content": input
    }
  ],
  temperature=1,
  max_tokens=tok,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [ ]:
response['choices'][0]['message']['content']

'In a tragic turn of events, the sailing community is grieving the loss of a crewman in the prestigious Volvo Ocean Race. The sailor, who was competing in the around-the-world race, was washed overboard and is now believed to be lost at sea.\n\nThe incident occurred during a particularly treacherous leg of the race, with harsh weather conditions and relentless waves posing a significant challenge to the competing boats. The crewman, whose identity has not been disclosed, was involved in a routine task onboard when an unexpected wave swept him overboard.\n\nImmediately following the incident, a frantic search and rescue operation was launched by the race organizers and fellow competitors. Despite their best efforts, however, the sailor could not be located, and it is now feared that he has succumbed to the harsh elements of the open sea.\n\nThe tragic news has sent shockwaves throughout the sailing community, as well as among the organizers and supporters of the Volvo Ocean Race. This p

In [ ]:
import openai
import os
from dotenv import load_dotenv
load_dotenv()
token = os.environ.get("api-token")
openai.api_key = token

In [ ]:
max_retries = 5
for x in range(1500):
    retries = 0
    samp = news.sample()
    news = news.drop(samp.index)
    while retries < max_retries:
        try:
            input = getinput(samp)
            tok = 150 + num_tokens_from_string(input,"gpt-3.5-turbo") + num_tokens_from_string(samp.iloc[0]['Article text'], "gpt-3.5-turbo")

            print("Generating......")

            response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k-0613",
            messages=[
                {
                "role": "system",
                "content": "You are a tool for generating article content given a tile, word length, and short description. You should make the article as close as you can to the specified length,preferably lower. You should only return the content of the article. If for some reason you cannot complete the request return the response \"Cannot complete request\" "
                },
                {
                "role": "user",
                "content": input
                }
            ],
            temperature=1,
            max_tokens=tok,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
            )
            samp['GPT3.5-Turbo'] = response['choices'][0]['message']['content']
            samp.to_csv('genArt.csv', mode='a', index=False, header=False)
            print(samp.iloc[0]['Headline'].split(" - ")[0])
            print(x)
            break  # Break out of the while loop if the API call is successful
        except Exception as e:
            retries += 1
            print(f"Error: {e}. Retrying...")

Generating......
Nike Oregon Project to shut down following Alberto Salazar ban
0
Generating......
Singer Kenny Chesney vows to help St. John after Irma
1
Generating......
Dog kills owner with forklift truck 
2
Generating......
Francisco Franco: Spanish dictator's remains can be exhumed says Spanish court
3
Generating......
Alexei Navalny says raids have targeted Russian opposition activists after Moscow poll
4
Generating......
Japan signs deal with Turkey to build nuclear plant
5
Generating......
Yvette Lundy, French Resistance heroine and concentration camp survivor, dead at 103
6
Generating......
Embassy security officer kidnapped in Yemen, Italy says
7
Generating......
An Olympic engagement story
8
Generating......
World Cup draw: Which group will your nation be drawn in for Russia 2018?
9
Generating......
A circus bear lunged at his handler with families sitting just feet away
10
Generating......
Alex Salmond, Scotland's ex-first minister, charged with attempted rape
11
Generating